---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# EXAMPLE DATA

In [9]:
# Date Range (input a year)
start_yr = 1980
stop_yr  = 2023

# Characteristic for second sort (1st is market equity)
key = "Book to market ratio"

# Some choices: 
# "Book to market ratio", "Investment rate", "Momentum", "Short term reversal",
# "Long term reversal", "Accruals", "Beta", "Net equity issuance", "Variance", "Residual variance"

# GET DATA

In [10]:
import numpy as np
import pandas as pd
from pandas_datareader import DataReader as pdr
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# associate names of French files with characteristic names
files = {'Earnings to price ratio': 'Portfolios_Formed_on_E-P',\
 'Variance': 'Portfolios_Formed_on_VAR',\
 'Accruals': 'Portfolios_Formed_on_AC',\
 'Residual variance': 'Portfolios_Formed_on_RESVAR',\
 'Net equity issuance': 'Portfolios_Formed_on_NI',\
 'Beta': 'Portfolios_Formed_on_BETA',\
 'Cash flow to price': 'Portfolios_Formed_on_CF-P',\
 'Market equity': 'Portfolios_Formed_on_ME',\
 'Book to market ratio': 'Portfolios_Formed_on_BE-ME',\
 'Dividend to price ratio': 'Portfolios_Formed_on_D-P',\
 'Investment rate': 'Portfolios_Formed_on_INV',\
 'Momentum': '10_Portfolios_Prior_12_2',\
 'Short term reversal': '10_Portfolios_Prior_1_0',\
 'Long term reversal': '10_Portfolios_Prior_60_13'}

# sort characteristics in alphabetical order
keys = np.sort(list(files.keys()))

# read value-weighted monthly returns in decimal format
f = files[key]
d = pdr(f,'famafrench',start=1920)[2]/100

# for momentum and reversal files, combine deciles to form quintiles
if 'Portfolios_Formed_on' not in f :
    cols = d.columns.to_list()
    d['Lo 20'] = d[cols[:2]].mean(axis=1)
    d['Qnt 2'] = d[cols[2:4]].mean(axis=1)
    d['Qnt 3'] = d[cols[4:6]].mean(axis=1)
    d['Qnt 4'] = d[cols[6:8]].mean(axis=1)
    d['Hi 20'] = d[cols[8:]].mean(axis=1)

# for other files, extract quintiles
else :    
    quintiles = ['Lo 20','Qnt 2','Qnt 3','Qnt 4','Hi 20']
    d = d[quintiles].copy()

d = d.reset_index()
d["Date"] = d.Date.astype(str).astype(int)
d = d.set_index("Date").dropna()
rets = d.copy()

# Subset for date window
mindate = max(start_yr, rets.index[0])
rets = rets.loc[mindate : stop_yr]

# CALCULATIONS

In [11]:
accum = (1 + rets).cumprod()
accum.loc[mindate - 1] = 1
accum = accum.sort_index()

rets = rets.stack().reset_index()
rets.columns = ["Date", "Quintile", "Return"]

accum = accum.stack().reset_index()
accum.columns = ["Date", "Quintile", "Accumulation"]

# FIGURE (CUMULATIVE RETURN)

In [12]:
import plotly.express as px


fig1 = px.line(accum, x="Date", y="Accumulation", color="Quintile")
string = "$%{y:,.2f}<extra></extra>"
fig1.update_traces(mode="lines", hovertemplate=string)
fig1.update_layout(
    xaxis_title="Date",
    yaxis_title="Compound Return",
    hovermode="x unified",
    legend = dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)
fig1.show()


# FIGURE (CUMULATIVE RETURN-LOG SCALE)

In [13]:
fig2 = px.line(accum, x="Date", y="Accumulation", color="Quintile", log_y=True)
fig2.update_traces(mode="lines", hovertemplate=string)
fig1.update_layout(
    xaxis_title="Date",
    yaxis_title="Compound Return (Log Scale)",
    hovermode="x unified",
    legend = dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)
fig2.show()

# FIGURE (DISTRIBUTIONS)

In [14]:
fig3 = px.box(rets, x="Quintile", y="Return", color="Quintile")
fig3.update_layout(
    xaxis_title="",
    yaxis_title="Return",
    yaxis_tickformat=".0%"
)
fig3.show()

# TABLE

In [15]:
rets = rets.set_index(["Date", "Quintile"]).unstack("Quintile")
data = rets.describe()["Return"].iloc[1:]
data.index.name = "Statistic"
data = data.reset_index()
data[['Statistic']+quintiles]

Quintile,Statistic,Lo 20,Qnt 2,Qnt 3,Qnt 4,Hi 20
0,mean,0.133907,0.131544,0.132251,0.127979,0.159812
1,std,0.199692,0.151969,0.157489,0.174382,0.199746
2,min,-0.349100,-0.347300,-0.346200,-0.486200,-0.351300
3,25%,0.022250,0.044000,0.039000,0.017150,0.015700
4,50%,0.126700,0.165000,0.151700,0.169400,0.219200
5,75%,0.303300,0.229850,0.229200,0.244800,0.298000
6,max,0.498000,0.389200,0.379300,0.367700,0.458800
